# Final Project: Human Action Recognition

In [ ]:
!unzip "Human Action Recognition.zip"

In [ ]:
import os
import glob
import numpy as np
import pandas as pd

from PIL import Image

import tensorflow as tf
from tensorflow import keras
import torch

import matplotlib.pyplot as plt

device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [ ]:
import tensorflow as tf
from tensorflow.python.keras import backend as K

config = tf.compat.v1.ConfigProto( device_count = {'GPU': 1 , 'CPU': 1} )
sess = tf.compat.v1.Session(config=config)
K.set_session(sess)

In [ ]:
# process the data
dataset = pd.read_csv("Human Action Recognition/Training_set.csv") # testing set does not have labels
folder = glob.glob("Human Action Recognition/train/*")
filename = dataset["filename"]
data_labels = dataset["label"]

images = []
labels = []
length = len(folder)
for i in range(length-1):
  img = Image.open("Human Action Recognition/train/" + filename[i])
  images.append(np.asarray(img.resize((160,160))))
  labels.append(data_labels[i])



In [ ]:
images_np = np.asarray(images)
labels_num, key = pd.factorize(np.asarray(data_labels))

y_train = keras.utils.to_categorical(np.asarray(labels_num))

#split dataset into training and validation
images_train = images_np[:10000]
images_validate = images_np[10000:]

labels_train = y_train[:10000]
labels_validate = y_train[10000:]


In [ ]:
# Specify Basic CNN model
CNN_basic_model = tf.keras.models.Sequential()
CNN_basic_model.add(keras.layers.Conv2D(filters = 32, kernel_size = (3,3), activation = 'relu', input_shape = (160,160,3)))
CNN_basic_model.add(keras.layers.MaxPool2D(pool_size=2, strides=2))
CNN_basic_model.add(keras.layers.Flatten())
CNN_basic_model.add(keras.layers.Dense(128, activation = 'relu'))
CNN_basic_model.add(keras.layers.Dense(15, activation = 'softmax'))

CNN_basic_model.compile(optimizer='adam', loss='categorical_crossentropy',metrics=['accuracy'])

In [ ]:
history1 = CNN_basic_model.fit(images_train, labels_train, validation_data = (images_validate, labels_validate), shuffle = True, epochs = 50)

In [ ]:
loss = history1.history["loss"]
val_loss = history1.history["val_loss"]
plt.title('Loss History')
plt.plot(loss, label = "training")
plt.plot(val_loss, label = "validation")
plt.xlabel("Epoch")
plt.ylabel("Loss")
plt.legend()

In [ ]:
accuracy = history1.history["accuracy"]
val_accuracy = history1.history["val_accuracy"]
plt.title('Accuracy History')
plt.plot(accuracy, label = "training")
plt.plot(val_accuracy, label = "validation")
plt.xlabel("Epoch")
plt.ylabel("Accuracy")
plt.legend()

In [ ]:
# Specify Improved CNN model
CNN_v2 = tf.keras.models.Sequential()
CNN_v2.add(keras.layers.Conv2D(filters = 32, kernel_size = (3,3), activation = 'relu', input_shape = (160,160,3)))
CNN_v2.add(keras.layers.MaxPool2D(pool_size=2, strides=2))
CNN_v2.add(keras.layers.Conv2D(filters = 32, kernel_size = (3,3), activation = 'relu', input_shape = (160,160,3)))
CNN_v2.add(keras.layers.MaxPool2D(pool_size=2, strides=2))
CNN_v2.add(keras.layers.Flatten())
CNN_v2.add(keras.layers.Dense(512, activation = 'relu'))
CNN_v2.add(keras.layers.Dropout(0.2))
CNN_v2.add(keras.layers.Dense(256, activation = 'relu'))
CNN_v2.add(keras.layers.Dropout(0.2))
CNN_v2.add(keras.layers.Dense(15, activation = 'softmax'))

CNN_v2.compile(optimizer=tf.keras.optimizers.Adam(), loss='categorical_crossentropy',metrics=['accuracy'])

In [ ]:
history3 = CNN_v2.fit(images_train, labels_train, validation_data = (images_validate, labels_validate), shuffle = True, epochs = 50)

In [ ]:
loss = history3.history["loss"]
val_loss = history3.history["val_loss"]
plt.title('Loss History')
plt.plot(loss, label = "training")
plt.plot(val_loss, label = "validation")
plt.xlabel("Epoch")
plt.ylabel("Loss")
plt.legend()

In [ ]:
accuracy = history3.history["accuracy"]
val_accuracy = history3.history["val_accuracy"]
plt.title('Accuracy History')
plt.plot(accuracy, label = "training")
plt.plot(val_accuracy, label = "validation")
plt.xlabel("Epoch")
plt.ylabel("Accuracy")
plt.legend()

In [ ]:
# Specify Improved CNN model with data augmentation
CNN_v3 = tf.keras.models.Sequential()
CNN_v3.add(keras.layers.Conv2D(filters = 32, kernel_size = (3,3), activation = 'relu', input_shape = (160,160,3)))
CNN_v3.add(keras.layers.MaxPool2D(pool_size=2, strides=2))
CNN_v3.add(keras.layers.Conv2D(filters = 32, kernel_size = (3,3), activation = 'relu', input_shape = (160,160,3)))
CNN_v3.add(keras.layers.MaxPool2D(pool_size=2, strides=2))
CNN_v3.add(keras.layers.Conv2D(filters = 64, kernel_size = (3,3), activation = 'relu', input_shape = (160,160,3)))
CNN_v3.add(keras.layers.MaxPool2D(pool_size=2, strides=2))
CNN_v3.add(keras.layers.Conv2D(filters = 64, kernel_size = (3,3), activation = 'relu', input_shape = (160,160,3)))
CNN_v3.add(keras.layers.MaxPool2D(pool_size=2, strides=2))
CNN_v3.add(keras.layers.Dropout(0.2))
CNN_v3.add(keras.layers.Flatten())
CNN_v3.add(keras.layers.Dense(512, activation = 'relu'))
CNN_v3.add(keras.layers.Dense(256, activation = 'relu'))
CNN_v3.add(keras.layers.Dense(64, activation = 'relu'))
CNN_v3.add(keras.layers.Dense(15, activation = 'softmax'))

CNN_v3.compile(optimizer='adam', loss='categorical_crossentropy',metrics=['accuracy'])

In [ ]:
datagen = keras.preprocessing.image.ImageDataGenerator(rotation_range=30, width_shift_range=0.1, height_shift_range=0.1)

In [ ]:
batch_size = 32
train_generator = datagen.flow(images_train, labels_train, batch_size=batch_size)

In [ ]:
history4 = CNN_v3.fit(train_generator, validation_data = (images_validate, labels_validate), shuffle = True, epochs = 50)

In [ ]:
loss = history4.history["loss"]
val_loss = history4.history["val_loss"]
plt.title('Loss History')
plt.plot(loss, label = "training")
plt.plot(val_loss, label = "validation")
plt.xlabel("Epoch")
plt.ylabel("Loss")
plt.legend()

In [ ]:
accuracy = history4.history["accuracy"]
val_accuracy = history4.history["val_accuracy"]
plt.title('Accuracy History')
plt.plot(accuracy, label = "training")
plt.plot(val_accuracy, label = "validation")
plt.xlabel("Epoch")
plt.ylabel("Accuracy")
plt.legend()

In [ ]:
# Specify VGG model
vgg16 = tf.keras.models.Sequential()

vgg_pretrained = tf.keras.applications.VGG16(include_top = False, input_shape = (160,160, 3), pooling = 'avg', classes = len(key), weights = "imagenet")

for layer in vgg_pretrained.layers:
  layer.trainable = False

vgg16.add(vgg_pretrained)
vgg16.add(keras.layers.Flatten())
vgg16.add(keras.layers.Dense(512, activation = 'relu'))
vgg16.add(keras.layers.Dense(256, activation = 'relu'))
vgg16.add(keras.layers.Dense(15, activation = 'softmax'))

vgg16.compile(optimizer='adam', loss='categorical_crossentropy',metrics=['accuracy'])




In [ ]:
history = vgg16.fit(images_train, labels_train, validation_data = (images_validate, labels_validate), shuffle = True, epochs = 50)

In [ ]:
loss = history.history["loss"]
val_loss = history.history["val_loss"]
plt.title('Loss History')
plt.plot(loss, label = "training")
plt.plot(val_loss, label = "validation")
plt.xlabel("Epoch")
plt.ylabel("Loss")
plt.legend()

In [ ]:
accuracy = history.history["accuracy"]
val_accuracy = history.history["val_accuracy"]
plt.title('Accuracy History')
plt.plot(accuracy, label = "training")
plt.plot(val_accuracy, label = "validation")
plt.xlabel("Epoch")
plt.ylabel("Accuracy")
plt.legend()

In [ ]:
#resnet50
resnet50 = tf.keras.models.Sequential()

resnet50_pretrained = tf.keras.applications.ResNet50(include_top = False, input_shape = (160,160, 3), pooling = 'avg', classes = len(key), weights = "imagenet")

for layer in resnet50_pretrained.layers:
  layer.trainable = False

resnet50.add(resnet50_pretrained)
resnet50.add(keras.layers.Flatten())
resnet50.add(keras.layers.Dense(512, activation = 'relu'))
resnet50.add(keras.layers.Dense(15, activation = 'softmax'))

resnet50.compile(optimizer='adam', loss='categorical_crossentropy',metrics=['accuracy'])


In [ ]:
history2 = resnet50.fit(images_train, labels_train, validation_data = (images_validate, labels_validate), shuffle = True, epochs = 50)

In [ ]:
loss = history2.history["loss"]
val_loss = history2.history["val_loss"]
plt.title('Loss History')
plt.plot(loss, label = "training")
plt.plot(val_loss, label = "validation")
plt.xlabel("Epoch")
plt.ylabel("Loss")
plt.legend()

In [ ]:
accuracy = history2.history["accuracy"]
val_accuracy = history2.history["val_accuracy"]
plt.title('Accuracy History')
plt.plot(accuracy, label = "training")
plt.plot(val_accuracy, label = "validation")
plt.xlabel("Epoch")
plt.ylabel("Accuracy")
plt.legend()

In [ ]:
# Specify VGG model with data augmentation
vgg16_aug = tf.keras.models.Sequential()

vgg_pretrained = tf.keras.applications.VGG16(include_top = False, input_shape = (160,160, 3), pooling = 'avg', classes = len(key), weights = "imagenet")

for layer in vgg_pretrained.layers:
  layer.trainable = False

vgg16_aug.add(vgg_pretrained)
vgg16_aug.add(keras.layers.Flatten())
vgg16_aug.add(keras.layers.Dense(512, activation = 'relu'))
vgg16_aug.add(keras.layers.Dense(256, activation = 'relu'))
vgg16_aug.add(keras.layers.Dense(15, activation = 'softmax'))

vgg16_aug.compile(optimizer='adam', loss='categorical_crossentropy',metrics=['accuracy'])


In [ ]:
history5 = vgg16_aug.fit(train_generator, validation_data = (images_validate, labels_validate), shuffle = True, epochs = 50)

In [ ]:
loss = history5.history["loss"]
val_loss = history5.history["val_loss"]
plt.title('Loss History')
plt.plot(loss, label = "training")
plt.plot(val_loss, label = "validation")
plt.xlabel("Epoch")
plt.ylabel("Loss")
plt.legend()

In [ ]:
accuracy = history5.history["accuracy"]
val_accuracy = history5.history["val_accuracy"]
plt.title('Accuracy History')
plt.plot(accuracy, label = "training")
plt.plot(val_accuracy, label = "validation")
plt.xlabel("Epoch")
plt.ylabel("Accuracy")
plt.legend()

In [ ]:
# resnet50 with data augmentation
resnet50_aug = tf.keras.models.Sequential()

resnet50_pretrained = tf.keras.applications.ResNet50(include_top = False, input_shape = (160,160, 3), pooling = 'avg', classes = len(key), weights = "imagenet")

for layer in resnet50_pretrained.layers:
  layer.trainable = False

resnet50_aug.add(resnet50_pretrained)
resnet50_aug.add(keras.layers.Flatten())
resnet50_aug.add(keras.layers.Dense(512, activation = 'relu'))
resnet50_aug.add(keras.layers.Dense(15, activation = 'softmax'))

resnet50_aug.compile(optimizer='adam', loss='categorical_crossentropy',metrics=['accuracy'])


In [ ]:
history6 = resnet50_aug.fit(train_generator, validation_data = (images_validate, labels_validate), shuffle = True, epochs = 50)

In [ ]:
loss = history6.history["loss"]
val_loss = history6.history["val_loss"]
plt.title('Loss History')
plt.plot(loss, label = "training")
plt.plot(val_loss, label = "validation")
plt.xlabel("Epoch")
plt.ylabel("Loss")
plt.legend()

In [ ]:
accuracy = history6.history["accuracy"]
val_accuracy = history6.history["val_accuracy"]
plt.title('Accuracy History')
plt.plot(accuracy, label = "training")
plt.plot(val_accuracy, label = "validation")
plt.xlabel("Epoch")
plt.ylabel("Accuracy")
plt.legend()